___
# Ciência dos Dados - PROJETO 2

___
## Nome: Gabriel Brey Gomes 
___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [75]:
import pandas as pd
import os
import numpy as np

In [76]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Usuário\Documents\#insper#\CD-DP\projeto 2


In [77]:
dados = pd.read_excel('spamhamV2.xlsx')


In [78]:
dados = dados.sample(frac=1)
treinamento = int(5572*0.75)
teste = int(5572*0.25)

dados_treinamento = dados.iloc[0:treinamento] 
dados_treinamento = dados_treinamento.reset_index(drop=True)

dados_teste = dados.iloc[treinamento:5572] 
dados_teste = dados_teste.reset_index(drop=True)




___
## 3. Classificador Naive-Bayes

In [79]:
def remover_caracteres(nome):
    nome = nome.replace('.', '').replace(':', ' ').replace('"', ' ').replace(')', '').replace('(', '') 
    return nome

In [80]:
dados_treinamento['Email'] = dados_treinamento['Email'].apply(remover_caracteres)


dados_teste['Email'] = dados_teste['Email'].apply(remover_caracteres)


In [81]:
Cemail = pd.Series(np.sum(dados_treinamento.Email + " ").split())
Cclass = pd.Series(np.sum(dados_treinamento.Class + " ").split())

In [82]:
prob = Cclass.value_counts('spam')
p_Spam = prob.iloc[1]
p_Ham = prob.iloc[0]

print('probabilidade de ser Spam : {0}'.format(p_Spam))
print('probabilidade de Não ser Spam : {0}'.format(p_Ham))

probabilidade de ser Spam : 0.13519980856664274
probabilidade de Não ser Spam : 0.8648001914333573


In [83]:
dadosL_Spam = dados_treinamento[dados_treinamento.Class == 'spam']
dadosL_nSpam = dados_treinamento[dados_treinamento.Class == 'ham']


In [84]:
lista_Spam = np.sum(dadosL_Spam.Email + " ").split()
S = pd.Series(lista_Spam)
lista_nSpam = np.sum(dadosL_nSpam.Email + " ").split()
H = pd.Series(lista_Spam)
TSpam = len(lista_Spam)
TnSpam = len(lista_nSpam)

In [85]:
def classifica(a):
    f = a.split(' ')
    pSpam = []
    pHam = []
    for i in range(len(f)):    
        pSpam.append((lista_Spam.count(f[i])+1)/(TSpam+len(f)))
        pHam.append((lista_nSpam.count(f[i])+1)/(TnSpam+len(f)))
    PSpam = np.prod(pSpam)
    PHam = np.prod(pHam)
    if PSpam > PHam:
        return "spam"
    else:
        return "ham"

In [86]:
T = []
for f in range(len(dados_teste.Email)):
  T.append(classifica(dados_teste.Email[f]))
dados_teste["resultado"] = pd.Series(T)

In [87]:
dados_teste.loc[(dados_teste.Class == "spam") & (dados_teste.resultado == "spam"),'verificação'] = 'N-V'
dados_teste.loc[(dados_teste.Class == "ham") & (dados_teste.resultado == "ham"),'verificação'] = 'P-V'
dados_teste.loc[(dados_teste.Class == "spam") & (dados_teste.resultado == "ham"),'verificação'] = 'P-F'
dados_teste.loc[(dados_teste.Class == "ham") & (dados_teste.resultado == "spam"),'verificação'] = 'N-F'

Negativo_Verdadeiro = dados_teste.verificação.value_counts()['N-V']
Positivo_Verdadeiro = dados_teste.verificação.value_counts()['P-V']
Negativo_Falso = dados_teste.verificação.value_counts()['N-F']
Positivo_Falso = dados_teste.verificação.value_counts()['P-F']

print("positivo falso:{0}".format(Positivo_Falso))
print("positivo verdadeiro:{0}".format(Positivo_Verdadeiro))
print("negativo verdadeiro:{0}".format(Negativo_Verdadeiro))
print("negativo falso: {0}".format(Negativo_Falso ))

positivo falso:4
positivo verdadeiro:1005
negativo verdadeiro:178
negativo falso: 206


In [88]:
print("positivo falso: {0}%".format((Positivo_Falso/(len(dados_teste))*100)))
print("positivo verdadeiro: {0}%".format((Positivo_Verdadeiro/(len(dados_teste))*100)))
print("negativo verdadeiro: {0}%".format((Negativo_Verdadeiro/(len(dados_teste))*100)))
print("negativo falso: {0}%".format((Negativo_Falso/(len(dados_teste)))*100))

positivo falso: 0.2871500358937545%
positivo verdadeiro: 72.14644651830582%
negativo verdadeiro: 12.778176597272076%
negativo falso: 14.788226848528357%


Concluímos que nosso teste é razoável, chegando a 75% de acerto, positivos falsos são inferior a 1% o que é bom pois o usuário perde o mínimo de e-mails Ham, mas em negativos falsos chegou a 14.5% que faz com que o usuário receba mais Spams. 

___
## 4. Curva ROC